In [1]:
print('Importing...')

import glob
import os
import numpy as np
import time as systime
import re 
import textgrid as tg

from scipy.stats import multivariate_normal

from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from ipyparallel import Client
import _pickle as pickle

import sys

print('success')

Importing...
success


- Manually specify a top string, a bottom string, and an MFCC output.

In [2]:
top_string = [0,1,0]
top_strings = [top_string]
bot_string = [0,2,1,0]
bot_string_states = [0,0,1,2, 6,7,7,8, 3,3,4,5, 0,1,2,2]
print('success')

success


Let's define the HMM/GMM parameters

In [3]:
# transition = [[.5, .5, 0],
#              [0, .5, .5],
#              [.75, 0, .25]]

components = []
# Distribution over components for each bottom PLU state
components.extend([[.7,.3],
             [.9,.1],
             [.3,.7]])
components.extend([[.3,.7],
             [.5,.5],
             [.8,.2]])
components.extend([[.4,.6],
             [.2,.8],
             [.9,.1]])

# We need a set of parameters for each component
# for each HMM state for each bottom PLU
# So in this case, we need 2 * 3 * 3 = 18 sets of parameters

component_parameters = [
        [
            [[0,0], [[1,0],[0,1]]], # bottom PLU 0, HMM state 0, component 0
            [[0.1,0.1], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 0, component 1
        ],
        [
             [[0.2,0.2], [[1,0],[0,1]]], # bottom PLU 0, HMM state 1, component 0
             [[0.3,0.3], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 1, component 1
        ],
        [
            [[0.4,0.4], [[1,0],[0,1]]], # bottom PLU 0, HMM state 2, component 0
            [[0.5,0.5], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 2, component 1
        ],
    
    
        [
            [[1.0,1.0], [[1,0],[0,1]]], # bottom PLU 1, HMM state 0, component 0
            [[1.1,1.1], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 0, component 1
        ],
        [
             [[1.2,1.2], [[1,0],[0,1]]], # bottom PLU 1, HMM state 1, component 0
             [[1.3,1.3], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 1, component 1
        ],
        [
            [[1.4,1.4], [[1,0],[0,1]]], # bottom PLU 1, HMM state 2, component 0
            [[1.5,1.5], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 2, component 1
        ],
    
          
        [
            [[2.0,2.0], [[1,0],[0,1]]], # bottom PLU 2, HMM state 0, component 0
            [[2.1,2.1], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 0, component 1
        ],
        [
             [[2.2,2.2], [[1,0],[0,1]]], # bottom PLU 2, HMM state 1, component 0
             [[2.3,2.3], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 1, component 1
        ],
        [
            [[2.4,2.4], [[1,0],[0,1]]], # bottom PLU 2, HMM state 2, component 0
            [[2.5,2.5], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 2, component 1
        ]
    

]


densities = [ [multivariate_normal(mean=comp[0], cov=comp[1]) for comp in comps] for comps in component_parameters]

densities_flat = []

[densities_flat.extend(x) for x in densities]

print('success')

print(densities_flat)


success
[<scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742b860>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742ba20>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bb00>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bba8>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bc50>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bcf8>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bda0>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742be48>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bef0>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bf98>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x11742bfd0>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x117447128>, <scipy.stats._multivariate.multivariate_normal_frozen o

In [4]:
print('running...')

def sample_gmm(bottom_hmm_states):
    
    mfccs = []
    
    for state in bottom_hmm_states:
        vec = None
        gmm_component_choice = np.random.choice(2,p=components[state])
        vec = np.random.multivariate_normal(*component_parameters[state][gmm_component_choice])
        mfccs.append(vec)
            
    return np.array(mfccs)

all_data = []


mfccs = sample_gmm(bot_string_states)

all_data.append(mfccs)

print('success')
        
print('bottom string: \n', bot_string_states)

print('mfccs: \n', mfccs)


for frame in mfccs:
    print([np.log(mv.pdf(frame)) for mv in densities_flat])
        

running...
success
bottom string: 
 [0, 0, 1, 2, 6, 7, 7, 8, 3, 3, 4, 5, 0, 1, 2, 2]
mfccs: 
 [[ 0.95803284 -0.95802781]
 [-0.45371188  0.18042325]
 [ 1.29655218 -0.43710816]
 [ 1.16795438 -0.42215481]
 [ 2.31433173  1.74371161]
 [ 1.24590564  0.31570672]
 [ 3.32538445  3.3557638 ]
 [ 1.88641814  1.23789534]
 [ 1.52310708  2.17672293]
 [-0.01754856  0.72273145]
 [ 0.38451475  1.01749945]
 [ 1.85650415  1.13590881]
 [-0.84570173  1.37126042]
 [-0.58927055 -0.30043477]
 [ 1.37966498  0.68456706]
 [ 1.02660971  1.49830079]]
[-2.75569917512962, -2.7656986718367293, -2.7956981685438387, -2.845697665250948, -2.9156971619580574, -3.0056966586651672, -3.755694142200714, -3.9656936389078239, -4.195693135614933, -4.445692632322042, -4.7156921290291516, -5.0056916257362616, -6.7556891092718079, -7.165688605978918, -7.5956881026860277, -8.0456875993931352, -8.515687096100244, -9.0056865928073542]
[-1.957080576867966, -1.994409440657448, -2.0517383044469302, -2.129067168236412, -2.2263960320258942,

In [5]:
def collect_data_stats(data):
    """Job to collect the statistics."""
    # We  re-import this module here because this code will run
    # remotely.
    
    stats_0 = data.shape[0]
    stats_1 = data.sum(axis=0)
    stats_2 = (data**2).sum(axis=0)
    retval = (
        stats_0,
        stats_1,
        stats_2
    )
    return retval

data_stats = list(map(collect_data_stats, all_data))

print('success')

success


In [6]:

def accumulate_stats(data_stats):
    n_frames = data_stats[0][0]
    mean = data_stats[0][1]
    var = data_stats[0][2]
    for stats_0, stats_1, stats_2 in data_stats[1:]:
        n_frames += stats_0
        mean += stats_1
        var += stats_2
    mean /= n_frames
    var = (var / n_frames) - mean**2

    data_stats = {
        'count': n_frames,
        'mean': mean,
        'var': var
    }
    return data_stats

final_data_stats = accumulate_stats(data_stats)

print('success')

success


In [7]:
print("Running...")

import subprocess
import amdtk
num_tops = 4

print("Finished imports in ipython notebook.")

print(len(all_data))
print(len(top_strings))
all_data_and_tops = list(zip(all_data, top_strings))
#print(all_data_and_tops[0])

print("starting engines")
subprocess.Popen(['ipcluster', 'start',' --profile', 'default',' -n', '4', '--daemonize'])
subprocess.Popen(['sleep', '10']).communicate()


rc = Client(profile='default')
dview = rc[:]
print('Connected to', len(dview), 'jobs.')


elbo = []
time = []
def callback(args):
    elbo.append(args['lower_bound'])
    time.append(args['time'])
    print('elbo=' + str(elbo[-1]), 'time=' + str(time[-1]))
 

print("Creating phone loop model...")
conc = 0.1



model = amdtk.PhoneLoopNoisyChannel.create(
    n_units=5,  # number of acoustic units
    n_states=3,   # number of states per unit
    n_comp_per_state=3,   # number of Gaussians per emission
    n_top_units=num_tops, # size of top PLU alphabet
    mean=np.zeros_like(final_data_stats['mean']), 
    var=np.ones_like(final_data_stats['var']),
    max_slip_factor=.05
    #concentration=conc
)


print("Creating VB optimizer...")   
optimizer = amdtk.ToyNoisyChannelOptimizer(
    dview, 
    final_data_stats, 
    args= {'epochs': 10,
     'batch_size': 4,
     'lrate': 0.01,
     'pkl_path': "example_test",
     'log_dir': 'logs'},
    model=model,
    dir_path='dir'
)

print("Running VB optimization...")
begin = systime.time()
print("running with toy data")
print('len(all_data_and_tops[0])')
print(len(all_data_and_tops[0]))
optimizer.run(all_data_and_tops, callback)
end = systime.time()
print("VB optimization took ",end-begin," seconds.")

# fig1 = figure(
#     x_axis_label='time (s)', 
#     y_axis_label='ELBO',
#     width=400, 
#     height=400
# )
# x = np.arange(0, len(elbo), 1)
# fig1.line(x, elbo)
#show(fig1)

print("\nDECODING\n")

date_string = systime.strftime("textgrids_%Y-%m-%d_%H:%M")

# Need to change this according to 
samples_per_sec = 100

all_data_and_tops = list(zip(all_data, top_strings))

for data, tops in all_data_and_tops:

    # Normalize the data
    data_mean = np.mean(data)
    data_var = np.var(data)
    data = (data-data_mean)/np.sqrt(data_var)

    #result = model.decode(data, tops, state_path=False)
    #result_path = model.decode(data, tops, state_path=True)
    (result_intervals, edit_path) = model.decode(data, tops, phone_intervals=True, edit_ops=True)

    print("---")
    print("Phone sequence:")
    print(result_intervals)
    print(edit_path)
    

Running...
INSIDE INIT
SUCCESSFUL IMPORT IN INIT
Finished imports in ipython notebook.
1
1
starting engines
Connected to 4 jobs.
Creating phone loop model...
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
Creating VB optimizer...
Running VB optimization...
running with toy data
len(all_data_and_tops[0])
2
state_llh at 4.961211:
(16, 15)
over -100 for:: 



[0, 1, 0]
[array([-4.09319534, -3.8972342 , -4.98629773, -2.83872289, -3.34551048,
       -3.13376399, -2.83742587, -3.38251428, -3.41039878, -2.642

[-2.92559726 -2.93011039 -2.92989105 -2.92816134 -2.93056706 -2.93095488
 -2.92868609 -2.92568954 -2.91591222 -2.92943596 -2.93043498]
[-2.92691632 -2.92833242 -2.92998008 -2.92990941 -2.92940392 -2.92998031
 -2.92313088 -2.92928019 -2.92768992 -2.92757187 -2.92965278]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
elbo=-7.19738284756 time=0.17490100860595703
state_llh at 5.12408:
(16, 15)
over -100 for:: 



[0, 1, 0]
[array([ -6.71781491,  -6.05721039,  -7.87294516,  -3.15570583,
        -5.2999808 ,  -4.39070877,  -3.88823035,  -4.32044374,
        -4.89632825,  -3.4870375 ,  -6.0621322 , -12.06412089,
        -7.54650134,  -6.78155771,  -4.0670274 ]), array([ -7.68491532,  -7.29775027,  -6.88518066,  -2.98938994,
        -5.35519784,  -4.86719791,  -3.6812661 ,  

over -100 for:: 



[0, 1, 0]
[array([-10.28486929,  -9.21467283, -11.31172984,  -4.25688822,
        -7.99612747,  -6.53067229,  -5.75502608,  -6.29401594,
        -7.31487586,  -5.32378411,  -8.61109474, -15.71056855,
       -11.0104663 ,  -8.58862059,  -5.46240195]), array([-11.47039944, -10.76775913,  -9.93182842,  -4.08258025,
        -8.12281128,  -7.26615734,  -5.69184583,  -6.66501375,
        -7.29545132,  -6.39879   ,  -8.4050786 , -14.20468354,
       -11.35528076,  -6.86303527,  -5.03342203]), array([ -8.44439617,  -7.62516276,  -9.45563026,  -3.51165415,
        -6.50107248,  -5.3170777 ,  -4.60124603,  -5.10765594,
        -5.89857144,  -4.32070522,  -7.07421654, -13.41638435,
        -9.11968054,  -7.24010336,  -4.52411887]), array([ -8.65977816,  -7.83670056,  -9.59272882,  -3.53965712,
        -6.63088298,  -5.43121921,  -4.66691202,  -5.19906635,
        -5.99888209,  -4.43450988,  -7.16371173, -13.51748949,
        -9.29536658,  -7.21676038,  -4.54385915]), array([-3

Frames: 16   Pb types: 5   Pt types: 2   Pt indices: 3
end item total:  -97.143703583
checking indices:
{16}
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
total states: 2530
forward states: 1356
backward states: 2350
ends in forward:  15
ends in backward:  30
starts in forward:  11
starts in backward:  11
log op counts:
[array([ -98.52322917, -114.4164331 ,  -98.87571956, -101.48210271,
       -108.90858168, -108.1015571 , -113.68602894,  -96.99803644,
       -100.71632903, -108.17417818, -106.75366208]), array([ -99.77222563, -114.00727087, -101.79901513, -101.08712933,
       -108.50544505, -107.99618766, -112.40198215,  -99.16102981,
        -99.47503432, -106.90276134, -106.32984252]), array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]), array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf])]
log_op_counts_normalized:
[[ -1.37952559 -17.27272952  -1.73201598  -4.3383

Frames: 16   Pb types: 5   Pt types: 2   Pt indices: 3
end item total:  -206.986462465
checking indices:
{16}
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
total states: 2530
forward states: 1356
backward states: 2350
ends in forward:  15
ends in backward:  30
starts in forward:  11
starts in backward:  11
log op counts:
[array([-208.36076981, -251.81189119, -208.53944983, -220.67839043,
       -235.56353639, -235.71023639, -251.14201986, -206.6851393 ,
       -219.98770377, -234.89389167, -235.04128635]), array([-209.4282528 , -251.41379761, -211.2962426 , -220.28544841,
       -235.16659598, -235.31366011, -249.79239129, -209.02331899,
       -218.65667117, -233.54771746, -233.69596436]), array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]), array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf])]
log_op_counts_normalized:
[[ -1.37430735 -44.82542873  -1.55298737 -13.691

ValueError: too many values to unpack (expected 2)

In [8]:
elbo

[]